# Future Health - List

In [1]:
import sys
sys.path.insert(0, '../../../lib/')

import tabula
import numpy as np
import pandas as pd
import importlib
import re

import pdfexport
importlib.reload(pdfexport)

from pdfexport import *

## Import PDF

In [21]:
# Read pdf into DataFrame
df = tabula.read_pdf("CH_2018_Report_V01.pdf", pages='all', lattice=True)

#columns = [55, 152, 224, 265, 350, 423, 465, 508, 553, 592, 635, 696, 739]
#df = tabula.read_pdf("CH_2018_Report_V01.pdf", pages='all', lattice=False, columns=columns, guess=False, pandas_options={'header': None}, multiple_tables=True)
#df = pd.concat(df)

In [26]:
df.head(10)

,SCHEDULE2‐TEMPLATE,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14
0,Date of publication:30/06/2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,Full Name\r(Art 1.01),HCPs: City of\rPrincipal Practice\rHCOs: city ...,Country of\rPrincipal\rPractice\r(Schedule 1),rincipal Practice Address\rArt 3),Unique country\ridentifier OPTIONAL\r(Art 3),onations\rnd Grants\ro HCOs (Art\r.01.1.a),Contribution to costs of Events (Art\r3.01.1.b...,Fee for service and consultancy\r(Art 3.01.1 c...,NaN,OTAL OPTIONAL,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sponsorship\ragreements\rwith\rHCOs/third\rpar...,Registration\rFees,Travel &\rAccommodatio\rn,Fees,Related expenses\ragreed in the fee\rfor servi...,NaN,NaN,NaN
3,NaN,INDIVIDUAL NAMED DISCLOSURE‐one line per HCP(i...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,H\rC\rP\rs,"Abdelli, Imene",La,Switzerland,NaN,Rue de Chasseral 20,NaN,NaN,NaN,570.59,1254.10,NaN,NaN,NaN,1824.69
5,NaN,"Antwi, Kwadwo",Basel,Switzerland,NaN,Petersgraben 4,7601003357066,NaN,NaN,NaN,128.90,NaN,NaN,NaN,128.90
6,NaN,"Arici, Birsen",Liestal,Switzerland,NaN,Rathausstrasse 24,7601000718198,NaN,NaN,739.22,800.75,NaN,NaN,NaN,1539.97
7,NaN,"Beuschlein, Felix",Zürich,Switzerland,NaN,Rämistrasse 100,7601003912746,NaN,NaN,NaN,147.74,NaN,NaN,NaN,147.74
8,NaN,"Boussaha, Tarek",Montreux,Switzerland,NaN,Avenue du Casino 40,7601003378481,NaN,NaN,NaN,NaN,700.00,NaN,NaN,700.00
9,NaN,"Christ‐Crain, Mirjam",Basel,Switzerland,NaN,.,7601000625311,NaN,NaN,NaN,NaN,4734.77,1106.92,NaN,5841.69


## Format Table

In [58]:
df_export = df.copy()

#Rename Columns
df_export.columns = ['empty', 'name', 'location', 'country', 'empty1', 'address', 'uci', 'donations_grants', 'sponsorship', 'registration_fees','travel_accommodation', 'fees', 'related_expenses', 'empty2', 'total']



#Replace by str
df_export.loc[df_export['empty'].str.contains('H\rC\rP\rs', na=False), 'empty'] = np.nan
df_export.loc[df_export['empty'].str.contains('H\rC\rO\rs', na=False), 'empty'] = np.nan

#Shift
df_export['empty2'] = df_export['empty2'].fillna("")
df_export['total'] = df_export['total'].fillna("")
df_export[df_export['empty'].notna()] = df_export[df_export['empty'].notna()].shift(1, axis='columns')

#Shift where address empty
df_export.loc[df_export.empty1.notna(), 'total'] = df_export['empty2']
df_export.loc[df_export.empty1.notna(), 'empty2'] = df_export['related_expenses']
df_export.loc[df_export.empty1.notna(), 'related_expenses'] = df_export['fees']
df_export.loc[df_export.empty1.notna(), 'fees'] = df_export['travel_accommodation']
df_export.loc[df_export.empty1.notna(), 'travel_accommodation'] = df_export['registration_fees']
df_export.loc[df_export.empty1.notna(), 'registration_fees'] = df_export['sponsorship']
df_export.loc[df_export.empty1.notna(), 'sponsorship'] = df_export['donations_grants']
df_export.loc[df_export.empty1.notna(), 'donations_grants'] = df_export['uci']
df_export.loc[df_export.empty1.notna(), 'uci'] = df_export['address']
df_export.loc[df_export.empty1.notna(), 'address'] = df_export['empty1']

#Remove rows which have no values
df_export.loc[df_export['total'] == "", 'total'] = np.nan
df_export = df_export.dropna(subset=['total'], how='all')
df_export = df_export.reset_index(drop=True)

#Remove empty columns
df_export.drop(columns=['empty', 'empty1', 'empty2'], inplace=True)

#Convert to Numbers
df_export = cleanup_number(df_export)
df_export = amounts_to_number(df_export)

#Add Type
df_export = set_type_by_alphabetical_order(df_export)

#Revert Name
df_export = revert_name(df_export, ',')

#Remove Carination
df_export = remove_carination(df_export, " ")

#Add PLZ
df_export = add_plz(df_export)

#basic string conversion
df_export = basic_string_conversion(df_export)

#Export
export_list(df_export, 'future')

Total nicht Summe der Werte
saved


In [ ]:
#write_to_csv(df_export, 'tmp.csv')
#write_to_excel(df_export, 'tmp.xlsx', open=True)